## Experiment_Pipeline

In [ ]:
import os
import sys
import yaml
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%pip install wandb -qU
%matplotlib inline

# Get the current working directory
notebook_dir = notebook_dir = os.path.dirname(os.path.abspath("__file__"))  
project_dir = os.path.abspath(os.path.join(notebook_dir, '..')) 
if project_dir not in sys.path:
    sys.path.append(project_dir)

from ensemble_classifier import (get_transforms, load_data, split_data, set_seeds, 
                 verify_splits, verify_data, plot_species_grid,
                 verify_loader_transforms)
from ensemble_classifier.data_utils import ImagesDataset
from ensemble_classifier.models import build_resnet50_basic, build_efficientnet_v2_basic
from ensemble_classifier.train import setup_training, evaluate, train

Ensure your directory is set up properly:

In [ ]:
# !tree ../ -L 2


In [ ]:
# !tree ../data/ -L 2

### **Set up your experiment**

Copy this notebook. Rename it, but keep it in `notebooks/`. To update any settings, params, and/or hyperparams make a copy of `configs/default.yaml`, rename it and call your new `.yaml` below. Be sure to keep it in `configs/`

In [ ]:
# Locate the YAML file relative to the notebook's location
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# You need to update this path to your new .yaml file
config_path = os.path.join(notebook_dir, "../configs/default_cuda.yaml")

# Load the YAML file
with open(config_path, "r") as f:
    config = yaml.safe_load(f)
print(config)

In [ ]:
print(torch.__version__)
print(torch.backends.mps.is_available())
device = config["device"]
print(f"Running on device: {device}")

### **Build the datasets**

#### Load the data
Note: your data file should be hidden in the repo (.gitignore) but make sure to set it up locally like:

`wildlife/data/givens/test_features/[images...]`

`wildlife/data/givens/train_features/[images...]`

`wildlife/data/givens/train_features.csv`

`wildlife/data/givens/test_features.csv`

In [ ]:

train_features, test_features, train_labels, species_labels = load_data()

#### Augment Data

In [ ]:
# Get transforms
train_transforms, val_transforms = get_transforms(config)
# print(train_transforms)

#### Split into train and evaluation sets

We need to ensure that sites are mutually exclusive between the training and validation sets, meaning no site should appear in both sets. This ensures a proper stratification based on site.

In [ ]:
set_seeds(config["experiment"]["seed"])
X_train, X_val, y_train, y_val = split_data(
    train_features, train_labels, type='sites')

In [ ]:
# Helper function (Optional)
# verify_splits(X_train, y_train, X_val,  y_val)

#### Set up DataLoader

In [ ]:
set_seeds(config["experiment"]["seed"])

# Create datasets
train_dataset = ImagesDataset(
    features=X_train, 
    labels=y_train, 
    transform=train_transforms, 
    device=device)

val_dataset = ImagesDataset(
    features=X_val, 
    labels=y_val, 
    transform=val_transforms, 
    device=device)

if device=="cuda":
    pin=False
else:
    pin=True

# Create DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=config["train"]["batch_size"], 
    shuffle=True, 
    pin_memory=pin)

val_loader = DataLoader(
    val_dataset, 
    batch_size=config["train"]["batch_size"], 
    shuffle=False, 
    pin_memory=pin)

In [ ]:
# verify transformations in dataloaders (Optional)
# verify_loader_transforms(train_loader, title_type='train')
# verify_loader_transforms(val_loader, title_type='validate')

# set_seeds(config["experiment"]['seed'])

In [ ]:
# Print shapes for verification (Optional)
# print(f"Training set: {len(train_dataset)} samples")
# print(f"Validation set: {len(val_dataset)} samples")

### **Training**


#### Define the model
Note: If you build a new model, add it to `models.py` and update the block below. And update your `.yaml` config.

In [ ]:
set_seeds(config["experiment"]['seed'])
# model = build_efficientnet_v2_basic(
#     num_classes = config["model"]["num_classes"],
#     hidden_units1 = config["model"]["hidden_units1"],
#     dropout = config["model"]["dropout"] 
# )
# model = model.to(device)


pretrained_antalope_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_bird_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_blank_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_civet_genet_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_hog_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_leopard_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_monkey_prosimian_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_rodent_model = build_resnet50_basic(
    # num_classes = config["model"]["num_classes"],
    num_classes = 1,
    hidden_units1 = config["model"]["hidden_units1"],
    dropout = config["model"]["dropout"] 
)

pretrained_antalope_model.load_state_dict(torch.load("../pretrained_models/pretrained_antelope_duiker_model.pth"))
pretrained_bird_model.load_state_dict(torch.load("../pretrained_models/pretrained_bird_model.pth"))
pretrained_blank_model.load_state_dict(torch.load("../pretrained_models/pretrained_blank_model.pth"))
pretrained_civet_genet_model.load_state_dict(torch.load("../pretrained_models/pretrained_civet_genet_model.pth"))
pretrained_hog_model.load_state_dict(torch.load("../pretrained_models/pretrained_hog_model.pth"))
pretrained_leopard_model.load_state_dict(torch.load("../pretrained_models/pretrained_leopard_model.pth"))
pretrained_monkey_prosimian_model.load_state_dict(torch.load("../pretrained_models/pretrained_monkey_prosimian_model.pth"))
pretrained_rodent_model.load_state_dict(torch.load("../pretrained_models/pretrained_rodent_model.pth"))

class EnsembleClassifier(torch.nn.Module):

    def __init__(self, model_antalope, model_bird, model_blank, model_civet_genet, model_hog, model_leopard, model_monkey_prosimian, model_rodent, output_class_count = 8):
            super(EnsembleClassifier, self).__init__()

            self.model_antalope = model_antalope
            self.model_bird = model_bird
            self.model_blank = model_blank
            self.model_civet_genet = model_civet_genet
            self.model_hog = model_hog
            self.model_leopard = model_leopard
            self.model_monkey_prosimian = model_monkey_prosimian
            self.model_rodent = model_rodent
            

        
    def forward(self, X):


        with torch.no_grad():
            # out_antelope = F.softmax(self.model_antalope(X), dim=1)
            # out_bird = F.softmax(self.model_bird(X), dim=1)
            # out_blank = F.softmax(self.model_blank(X), dim=1)
            # out_civet_genet = F.softmax(self.model_civet_genet(X), dim=1)
            # out_hog = F.softmax(self.model_hog(X), dim=1)
            # out_leopard = F.softmax(self.model_leopard(X), dim=1)
            # out_monkey_prosimian = F.softmax(self.model_monkey_prosimian(X), dim=1)
            # out_rodent = F.softmax(self.model_rodent(X), dim=1)

            out_antelope = (self.model_antalope(X))
            out_bird = (self.model_bird(X))
            out_blank = (self.model_blank(X))
            out_civet_genet = (self.model_civet_genet(X))
            out_hog = (self.model_hog(X))
            out_leopard = (self.model_leopard(X))
            out_monkey_prosimian = (self.model_monkey_prosimian(X))
            out_rodent = (self.model_rodent(X))

    
        # print(f"out Antelope is: {out_antelope}")
        # print(f"out bird is: {out_bird}")
        # print(f"Out blank is: {out_blank}")
        # print(f"Out_civet genet: {out_civet_genet}")
        # print(f"outpu_hog: {out_hog}")
        # print(f"out leopard: {out_leopard}")
        # print(f"Out monkey: {out_monkey_prosimian}")
        # print(f"out rodent: {out_rodent}")
        
        #mega_X = torch.stack((out_antelope[:,1], out_bird[:,1], out_blank[:,1], out_civet_genet[:,1], out_hog[:,1], out_leopard[:,1], out_monkey_prosimian[:,1], out_rodent[:,1]), dim = 0).T
        
        mega_X = torch.stack((out_antelope, out_bird, out_blank, out_civet_genet, out_hog, out_leopard, out_monkey_prosimian, out_rodent), dim = 0)
        # print(f"Shape of mega X: {mega_X}")
        
        return mega_X


model = EnsembleClassifier(pretrained_antalope_model, pretrained_bird_model, pretrained_blank_model, pretrained_civet_genet_model, pretrained_hog_model, pretrained_leopard_model, pretrained_monkey_prosimian_model, pretrained_rodent_model, output_class_count=8)
model = model.to(device)

#### Define your criterion and optimizer
Note: If needed up date these in `train.py` and update your `.yaml` config.

In [ ]:
set_seeds(config["experiment"]['seed'])
criterion, optimizer = setup_training(
        model, 
        criterion=config["train"]["criterion"],
        optimizer=config["train"]["optimizer"], 
        lr=config["train"]["lr"], 
        momentum=config["train"]["momentum"])

#### Set up logging

In [ ]:
wandb.require()
wandb.login()

In [ ]:
# ✨ W&B: Initialize a new run to track this model's training
wandb.init(project="wildlife")
cfg = wandb.config
cfg.update(config)

#### Run the train / eval loop

In [27]:
log_counter = 0
tracking_loss_all = []
train_losses = []  # To store average training loss per epoch
val_losses = []    # To store validation loss per epoch
set_seeds(config["experiment"]['seed'])

start_time = time.time()
for epoch in range(config["train"]["epochs"]):
    # # Training step
    # avg_train_loss, tracking_loss = train(model, 
    #                                  train_loader, 
    #                                  criterion, 
    #                                  optimizer, 
    #                                  epoch, config, device=device)
    # tracking_loss_all.extend(tracking_loss)  # Append to global list
    # train_losses.append(avg_train_loss)  # Store avg training loss
    # print(f"Epoch {epoch+1}/{config['train']['epochs']} - Avg Train Loss: {avg_train_loss:.4f}")
    
    # Evaluation step
    eval_metrics = evaluate(model, val_loader, criterion, config, epoch= epoch+1, device=device)
    val_losses.append(eval_metrics["loss"])  # Store validation loss
    print(f"Epoch {epoch+1}/{config['train']['epochs']} - Eval Loss: {eval_metrics['loss']:.4f}, Eval Acc: {eval_metrics['accuracy']:.2f}%")

end_time = time.time()
duration = end_time - start_time
wandb.log({"duration": duration})


Loss: 5.185973167419434
Predicted: 
tensor([6, 3, 0, 4, 3, 6, 2, 2], device='cuda:0')
Correct:
tensor([1, 5, 6, 1, 6, 6, 6, 6], device='cuda:0')

Loss: 4.202817916870117
Predicted: 
tensor([0, 0, 5, 2, 1, 1, 7, 2], device='cuda:0')
Correct:
tensor([0, 4, 7, 0, 5, 0, 3, 6], device='cuda:0')

Loss: 8.101451873779297
Predicted: 
tensor([5, 2, 5, 7, 0, 7, 1, 6], device='cuda:0')
Correct:
tensor([4, 6, 1, 2, 5, 2, 7, 3], device='cuda:0')

Loss: 3.8915910720825195
Predicted: 
tensor([3, 2, 3, 5, 3, 5, 4, 0], device='cuda:0')
Correct:
tensor([7, 0, 0, 0, 1, 7, 3, 2], device='cuda:0')

Loss: 7.389688491821289
Predicted: 
tensor([1, 1, 7, 3, 5, 0, 2, 3], device='cuda:0')
Correct:
tensor([0, 1, 6, 7, 5, 0, 5, 2], device='cuda:0')

Loss: 3.1937551498413086
Predicted: 
tensor([1, 1, 6, 4, 0, 4, 2, 3], device='cuda:0')
Correct:
tensor([7, 7, 0, 7, 7, 5, 5, 0], device='cuda:0')

Loss: 3.2299718856811523
Predicted: 
tensor([2, 2, 5, 4, 0, 5, 5, 4], device='cuda:0')
Correct:
tensor([5, 5, 6, 2, 7, 5, 

ValueError: Expected input batch_size (8) to match target batch_size (5).

If you are done logging or you want to run the experiment again, finish with the block below. But if you think you might want to submit this run to the competition, don't finish logging until the end once you've added the competition score.

In [ ]:
# ✨ W&B: Mark the run as complete (Or wait until the end of notebook)
wandb.finish()

---


### **Explore Experiment** 

In [ ]:
# set to True to explore and potentially submit your results 
explore = True

#### Learning Curve

In [ ]:
if explore:
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, config["train"]["epochs"]+1), train_losses, label="Training Loss", marker="o")
    plt.plot(range(1, config["train"]["epochs"]+1), val_losses, label="Validation Loss", marker="o")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Learning Curve")
    plt.legend()
    plt.grid(True)
    plt.show()

#### Batch Loss During Training

In [ ]:
if explore:
    # Convert tracking_loss to a pandas Series for convenient rolling average
    tracking_loss_series = pd.Series(tracking_loss_all)

    # Plot
    plt.figure(figsize=(10, 5))
    tracking_loss_series.plot(alpha=0.2, label="Batch Loss")
    tracking_loss_series.rolling(center=True, min_periods=1, window=10).mean().plot(
        label="Loss (Moving Avg)", linewidth=2
    )
    plt.xlabel("(Epoch, Batch)")
    plt.ylabel("Loss")
    plt.title("Batch Loss During Training")
    plt.legend(loc="upper right")
    plt.show()

#### Class Distribution  

True Labels from Training Set

In [ ]:
if explore:
    print("True labels (training):")
    print(y_train.idxmax(axis=1).value_counts())   

True and Predicated Labels from Validation Set

In [ ]:
if explore:
    # Extract predictions and true labels from eval_metrics
    all_preds = eval_metrics["all_preds"]
    all_labels = eval_metrics["all_labels"]

    # Convert all_preds to DataFrame and map to class names
    preds_df = pd.DataFrame(all_preds, columns=["predicted_class"])
    preds_df["predicted_label"] = preds_df["predicted_class"].map(
        lambda idx: species_labels[idx]
    )

    # Convert all_labels to DataFrame and map to class names
    labels_df = pd.DataFrame(all_labels, columns=["true_class"])
    labels_df["true_label"] = labels_df["true_class"].map(
        lambda idx: species_labels[idx]
    )

    # Combine predictions and true labels for analysis
    results_df = pd.concat([preds_df, labels_df], axis=1)

    # Display value counts for predicted and true labels
    print("Predicted labels (eval):")
    print(results_df["predicted_label"].value_counts())

    print("\nTrue labels (eval):")
    print(results_df["true_label"].value_counts())

Accuracy per class

In [ ]:
if explore:    
    per_class_accuracy = results_df.groupby("true_label").apply(
        lambda x: (x["true_label"] == x["predicted_label"]).mean(), 
    )
    print("Per-Class Accuracy:")
    print(per_class_accuracy)

Confusion Matrix

In [ ]:
if explore:
    from sklearn.metrics import ConfusionMatrixDisplay

    eval_true = pd.Series(all_labels).apply(lambda x: species_labels[x])
    eval_predictions = pd.Series(all_preds).apply(lambda x: species_labels[x])

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(10, 10))
    cm = ConfusionMatrixDisplay.from_predictions(
        eval_true,
        eval_predictions,
        ax=ax,
        xticks_rotation=90,
        colorbar=True,
        normalize='true'
    )
    plt.title("Normalized Confusion Matrix")
    plt.show()

### **Create Submission**

### Set up Datatloader

In [ ]:
if explore:
    set_seeds(config["experiment"]["seed"])
    test_dataset = ImagesDataset(
        test_features, 
        transform=val_transforms, 
        device=device)

    if(device=="cuda"):
        pin=False
    else:
        pin=True

    test_loader = DataLoader(
        test_dataset, 
        batch_size=config["train"]["batch_size"], 
        shuffle=False, pin_memory=pin)
    
    print(f"Test set: {len(test_dataset)} samples")

### Test Model

In [ ]:
if explore:
    test_preds_collector = []

    # put the model in eval mode so we don't update any parameters
    model.eval()

    # we aren't updating our weights so no need to calculate gradients
    with torch.no_grad():
        for batch_n, batch in enumerate(test_loader):
            # run the forward step
            images = batch["image"].to(device)
            logits = model(images)

            # apply softmax so that model outputs are in range [0,1]
            preds = F.softmax(logits, dim=1)

            # store this batch's predictions in df
            # note that PyTorch Tensors need to first be detached from their computational graph before converting to numpy arrays
            preds_df = pd.DataFrame(
                preds.cpu().numpy(),
                index=batch["image_id"],
                columns=species_labels,
            )
            test_preds_collector.append(preds_df)

    submission_df = pd.concat(test_preds_collector)

Create your submission. Update submission_number.

Make sure your directory is properly set up, as both `/data` and `/results` are ignored by the repo.

In [ ]:
if explore:
    submission_number=14
    submission_df.index.name = 'id'
    submission_df = submission_df.round(6)
    submission_format_path = "../data/givens/submission_format.csv"
    submission_format = pd.read_csv(submission_format_path, index_col="id")


    assert all(submission_df.index == submission_format.index)
    assert all(submission_df.columns == submission_format.columns)

    # Save submission_df for further use
    submission_df_path = f"../results/submissions/submission{submission_number}.csv"
    submission_df.to_csv(submission_df_path)

After you submit update the submission score for logging.

In [ ]:
if explore:
     # ✨ Mannualy Log Test Results to W&B
    wandb.log({
        "test_score": 1.4578
    })

End your logging session.

In [ ]:
if explore:
    # ✨ W&B: Mark the run as complete (Or wait until the end of notebook)
    wandb.finish()